In [1]:
import pandas as pd
import pickle as pkl 
import numpy as np

In [2]:
data = pd.read_csv('holdout_data.csv')

In [3]:
columns = ['credit_given', 'gender', 'education', 'marital_status', 'age', 'sep_pay', 'aug_pay', 'jul_pay', 'jun_pay', 'may_pay', 'apr_pay', 'sep_bill', 'aug_bill', 'jul_bill', 'jun_bill', 'may_bill', 'apr_bill', 'sep_paid', 'aug_paid', 'jul_paid', 'jun_paid', 'may_paid', 'apr_paid']

data.drop(columns = 'Unnamed: 0', inplace = True)

data.columns = columns

In [4]:
data = data[['credit_given', 'gender', 'education', 'marital_status', 'age', 'sep_pay', 'aug_pay', 'jul_pay', 'jun_pay', 'may_pay', 'apr_pay', 'sep_bill', 'aug_bill', 'jul_bill', 'jun_bill', 'may_bill', 'apr_bill', 'sep_paid', 'aug_paid', 'jul_paid', 'jun_paid', 'may_paid', 'apr_paid']].astype(int)

In [5]:
data.education = data.education.apply(lambda x: 4 if x > 3 or x == 0 else x)

data['gender'] = np.where(data.gender == 1, data.gender, 0)

In [7]:
data.apr_pay = data.apr_pay.apply(lambda x: 6 if x > 6 else x)
data.may_pay = data.may_pay.apply(lambda x: 6 if x > 6 else x)
data.jun_pay = data.jun_pay.apply(lambda x: 6 if x > 6 else x)
data.jul_pay = data.jul_pay.apply(lambda x: 6 if x > 6 else x)
data.aug_pay = data.aug_pay.apply(lambda x: 6 if x > 6 else x)
data.sep_pay = data.sep_pay.apply(lambda x: 6 if x > 6 else x)

data.apr_pay = data.apr_pay.apply(lambda x: 2 if x==1 else x)
data.may_pay = data.may_pay.apply(lambda x: 2 if x==1 else x)
data.jun_pay = data.jun_pay.apply(lambda x: 2 if x==1 else x)
data.jul_pay = data.jul_pay.apply(lambda x: 2 if x==1 else x)
data.aug_pay = data.aug_pay.apply(lambda x: 2 if x ==1 else x)
data.sep_pay = data.sep_pay.apply(lambda x: 2 if x ==1 else x)

In [8]:
data.apr_bill = data.apr_bill.apply(lambda x: 0 if x < 0 else x)
data.may_bill = data.may_bill.apply(lambda x: 0 if x < 0 else x)
data.jun_bill = data.jun_bill.apply(lambda x: 0 if x < 0 else x)
data.jul_bill = data.jul_bill.apply(lambda x: 0 if x < 0 else x)
data.aug_bill = data.aug_bill.apply(lambda x: 0 if x < 0 else x)
data.sep_bill = data.sep_bill.apply(lambda x: 0 if x < 0 else x)

In [9]:
data['avg_bill'] = data[['sep_bill', 'aug_bill', 'jul_bill', 'jun_bill', 'may_bill', 'apr_bill']].mean(axis = 1)
data['avg_paid'] = data[['sep_paid', 'aug_paid', 'jul_paid', 'jun_paid', 'may_paid', 'apr_paid']].mean(axis = 1)
data['avg_dif'] = data['avg_bill'] - data['avg_paid']

In [10]:
df_use = pd.DataFrame(columns = ['a'])

df_use['sep_use'] = data.sep_bill/data.credit_given
df_use['aug_use'] = data.aug_bill/data.credit_given
df_use['jul_use'] = data.jul_bill/data.credit_given
df_use['jun_use'] = data.jun_bill/data.credit_given
df_use['may_use'] = data.may_bill/data.credit_given
df_use['apr_use'] = data.apr_bill/data.credit_given
#added since last test
df_use['avg_use'] = df_use[['sep_use', 'aug_use', 'jul_use', 'jun_use', 'may_use', 'apr_use']].mean(axis = 1)


df_use.drop(columns = 'a', inplace = True)

In [11]:
def dummy_list(column, prefix, drop_value):
    ''' Takes in a column, a prefix and drop value and generates a DataFrame of dummy columns with the specified prefix and column categories minus the specified drop value'''
    dum = pd.get_dummies(column, prefix = prefix)
    dum = dum.drop(columns = f'{prefix}_{drop_value}')
    return dum

def dummy_df(list_of_columns, list_of_prefixes, list_of_drop_values):
    r = pd.DataFrame(columns = ['a'])
    for i in range(0, len(list_of_columns)-1):
        r = pd.concat([r, dummy_list(list_of_columns[i], list_of_prefixes[i], list_of_drop_values[i])], axis = 1)
    return r

In [12]:
cols = [data.sep_pay, data.aug_pay, data.jul_pay, data.jun_pay, data.may_pay, data.apr_pay]

pref = ['sep', 'aug', 'jul', 'jun', 'may', 'apr']

drops = [6,6,6,6,6,6]
#tune this parameter
dum_df = dummy_df(cols, pref, drops)

dum_df.drop(columns = 'a', inplace = True)

mar_dum = dummy_list(data.marital_status, 'mar', 0)

ed_dum = dummy_list(data.education, 'ed', 4)

data.drop(columns = ['sep_pay', 'aug_pay', 'jul_pay', 'jun_pay', 'may_pay', 'apr_pay', 'marital_status', 'education'], inplace = True)

In [13]:
X = pd.concat([data, dum_df, mar_dum, ed_dum], axis = 1)

In [14]:
model_pickle_path = 'bakeoff_model.pkl'

rfc_model_pkl = open(model_pickle_path, 'rb')
rfc_model = pkl.load(rfc_model_pkl)
print("Loaded RFC model :: ", rfc_model)

Loaded RFC model ::  RandomForestClassifier(class_weight='balanced', max_depth=6, max_features=11,
                       n_estimators=550, n_jobs=-1, oob_score=True,
                       random_state=1, verbose=1)


In [17]:
pd.DataFrame(rfc_model.predict(X)).to_csv('bakeoff_preds_Nick_Subic.csv')

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 550 out of 550 | elapsed:    0.2s finished


In [20]:
preds = rfc_model.predict(X)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 550 out of 550 | elapsed:    0.2s finished


In [22]:
pd.Series(preds).value_counts()

0    5576
1    1925
dtype: int64